# Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, RobustScaler, normalize
from sklearn.model_selection import train_test_split

DEBUG = True

# Read data

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sample = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
def plot_sample(sample_id, df):
    df_breath = df[df['breath_id'] == sample_id]
    r, c  = df_breath[['R', 'C']].values[0]

    cols = ['u_in', 'u_out', 'pressure'] if 'pressure' in df.columns else ['u_in', 'u_out']
    
    plt.figure(figsize=(12, 4))
    for col in ['u_in', 'u_out', 'pressure']:
        plt.plot(df_breath['time_step'], df_breath[col], label=col)
        
    plt.legend()
    plt.title(f'Sample {sample_id} - R={r}, C={c}')

In [ ]:
df = train[train['breath_id'] < 5].reset_index(drop=True)

In [ ]:
df.head()

# Visualize

In [ ]:
for i in df['breath_id'].unique():
    plot_sample(i, train)

In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
if DEBUG:
    train = train[:80*200]

In [ ]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

In [ ]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [ ]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [ ]:
train.shape

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Define Model

In [ ]:
# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model = tf.keras.Sequential([
    keras.layers.Input(shape=train.shape[-2:]),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])
    model.compile(optimizer='adam',
                steps_per_execution = 50,
                loss='mae',
                metrics=['accuracy'])

# Split data for validation

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, targets, test_size=0.33, random_state=42)

# Fit data

In [ ]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2
EPOCH = 5

In [ ]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid),epochs=EPOCH, batch_size=BATCH_SIZE)

# Predict

In [ ]:
test_preds = []
test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

In [ ]:
sample["pressure"] = sum(test_preds)
sample.to_csv('submission.csv', index=False)

References:

https://www.kaggle.com/theoviel/deep-learning-starter-simple-lstm
https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm

Thank you.